# Lecture 4: Simple Algorithms for Stock Return Prediction

## Introduction

In this notebook, we will implement and compare **four key machine learning models** for stock return prediction:

1. **Linear Regression (OLS)** – the baseline linear model.
2. **Regularized Regression (Lasso, Ridge, Elastic Net)** – improved generalization in high-dimensional, noisy settings.
3. **Random Forest (RF)** – nonlinear, ensemble method robust to noise.
4. **Gradient Boosting (GBDT, XGBoost, LightGBM)** – sequential ensemble method, industry standard for tabular prediction.

---

### Why Machine Learning for Stock Prediction?
- Stock returns are **noisy** and often weakly predictable.  
- Firm characteristics can be **high-dimensional** and correlated.  
- Machine learning models help capture **nonlinear patterns** and avoid overfitting via regularization and ensembles.

---

### Evaluation Metrics
We will compare models using the following metrics:

- **RMSE (Root Mean Squared Error):** accuracy, penalizes large errors.  
- **MAE (Mean Absolute Error):** average error magnitude, robust to outliers.  
- **Hit Ratio:** percentage of correct up/down predictions.  
- **Fit Time:** computational efficiency of model training.  

> In finance, **out-of-sample performance** is the gold standard — models must be evaluated on unseen data to ensure predictive power.

---

### Notebook Workflow
1. Load and preprocess financial data.  
2. Train models using historical features.  
3. Evaluate models with RMSE, MAE, Hit Ratio, and training time.  
4. Compare results to highlight **strengths and limitations** of each method.  

---


## 0. Using Pandas_datareader to Retrieve Factor Data

- **Pandas_datareader** is a Python library for accessing financial and economic data from various online sources.  
  It supports the following major databases:  

  - **Tiingo**: Provides historical price data for stocks, indices, and funds. (Requires an API key, free registration available.)  
  - **IEX (Investors Exchange)**: Offers real-time and historical data for the U.S. stock market (requires an API key).  
  - **Alpha Vantage**: Provides real-time U.S. stock and foreign exchange data (requires an API key).  
  - **Econdb**: Supplies macroeconomic data for major countries worldwide.  
  - **Yahoo Finance**: Offers historical price data for stocks, indices, and funds.  
  - **FRED (Federal Reserve Economic Data)**: Provides U.S. economic and financial time-series data.  
  - **World Bank**: Supplies macroeconomic and development data for countries worldwide.  
  - **OECD (Organisation for Economic Co-operation and Development)**: Provides economic and social data for member countries.  
  - **Eurostat**: Offers official statistics from the Statistical Office of the European Union.  
  - **Nasdaq Trader**: Provides stock data from the Nasdaq exchange.  
  - **Stooq**: Offers global data on stocks, indices, and foreign exchange.  
  - **TSP (Thrift Savings Plan)**: Provides data on the U.S. federal government employees’ retirement savings plan.  
  - **Fama-French**: Offers data for the Fama-French factor models, including market, size, and value factors.  

Since WRDS network restrictions prevent direct demonstration here, more details about Pandas_datareader can be found at:  
👉 https://pandas-datareader.readthedocs.io/en/latest/remote_data.html


In [166]:
# === Requirements / Imports ===
# Data handling
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning - scikit-learn
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler

import pandas_datareader.data as web

# Gradient Boosting - external libraries
from xgboost import XGBRegressor
import lightgbm as lgb

# Timing
import time

# Display settings
sns.set_style("whitegrid")
plt.rcParams["figure.figsize"] = (8, 5)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12


In [167]:
# === Data Loading from WRDS ===
import os
import wrds
from dotenv import load_dotenv

load_dotenv()
wrds_user = os.getenv("WRDS_USER")

# Connect to WRDS
db = wrds.Connection(wrds_username=wrds_user)

# ---------------------------
# 1. CRSP Monthly Stock Data
# ---------------------------
crsp_msf = db.raw_sql("""
    select a.permno, a.date, a.ret, a.vol, a.shrout, a.prc,
           b.ticker, b.comnam
    from crsp.msf as a
    left join crsp.msenames as b
    on a.permno=b.permno
    and b.namedt <= a.date
    and a.date <= b.nameendt
    where a.date between '2010-01-01' and '2020-12-31'
""")

# Clean CRSP
crsp_msf['date'] = pd.to_datetime(crsp_msf['date'])
crsp_msf['ret'] = pd.to_numeric(crsp_msf['ret'], errors='coerce')
crsp_msf['me'] = crsp_msf['prc'].abs() * crsp_msf['shrout']   # Market equity

# ---------------------------
# 2. Fama-French Factors (Monthly)
# ---------------------------
ff_factors = db.raw_sql("""
    select date, mktrf, smb, hml, rf
    from ff.factors_monthly
    where date between '2010-01-01' and '2020-12-31'
""")

# ---------------------------
# 3) Compustat Fundamentals (Quarterly) — build 10 features
# ---------------------------
# Pull key quarterly fields from comp.fundq
compq = db.raw_sql("""
    select gvkey, datadate,
           atq,        -- total assets (quarter)
           ceqq,       -- common equity (quarter)
           revtq,      -- revenue (total)
           cogsq,      -- cost of goods sold
           ibq,        -- income before extraordinary items
           dlttq,      -- long-term debt
           dlcq,       -- debt in current liabilities (short-term debt)
           capxy,      -- capital expenditures
           cheq,       -- cash & equivalents
           actq,       -- current assets
           lctq        -- current liabilities
    from comp.fundq
    where indfmt = 'INDL'
      and datafmt = 'STD'
      and popsrc = 'D'
      and consol = 'C'
      and datadate between '2009-01-01' and '2020-12-31'
""")

# Ensure datetime and reasonable ordering
compq['datadate'] = pd.to_datetime(compq['datadate'], errors='coerce')
compq = compq.sort_values(['gvkey', 'datadate']).rename(columns={"capxy": "capxq"})

# Convert to numeric (in case of strings), keep as floats
num_cols = ['atq','ceqq','revtq','cogsq','ibq','dlttq','dlcq','capxq','cheq','actq','lctq']
for c in num_cols:
    compq[c] = pd.to_numeric(compq[c], errors='coerce')

# Create a quarter-end "date" aligned to month-end (for later merges to monthly)
# Using PeriodIndex to guarantee quarter END timestamp
compq['date'] = pd.PeriodIndex(compq['datadate'], freq='Q').to_timestamp(how='end')
compq['date'] = compq['date'].dt.to_period('M').dt.to_timestamp('M')  # normalize to month-end

# Build features (protect against divide-by-zero with where/clip)
eps = 1e-12

# 1) Size (log of assets)
compq['size_log_at'] = np.log(compq['atq'].where(compq['atq'] > 0))

# 2) Book-to-Market (quarterly proxy): common equity / assets
compq['bm'] = (compq['ceqq'] / compq['atq']).replace([np.inf, -np.inf], np.nan)

# 3) Leverage: (long-term + short-term debt) / assets
compq['lev'] = ((compq['dlttq'].fillna(0) + compq['dlcq'].fillna(0)) / compq['atq']).replace([np.inf, -np.inf], np.nan)

# 4) Sales growth QoQ: (revtq / L1(revtq) - 1)
compq['revtq_l1'] = compq.groupby('gvkey')['revtq'].shift(1)
compq['sales_g_qoq'] = (compq['revtq'] / compq['revtq_l1'] - 1).replace([np.inf, -np.inf], np.nan)

# 5) Sales growth YoY: (revtq / L4(revtq) - 1)
compq['revtq_l4'] = compq.groupby('gvkey')['revtq'].shift(4)
compq['sales_g_yoy'] = (compq['revtq'] / compq['revtq_l4'] - 1).replace([np.inf, -np.inf], np.nan)

# 6) Asset growth YoY: (atq / L4(atq) - 1)
compq['atq_l4'] = compq.groupby('gvkey')['atq'].shift(4)
compq['asset_g_yoy'] = (compq['atq'] / compq['atq_l4'] - 1).replace([np.inf, -np.inf], np.nan)

# 7) Profitability (ROA): ibq / assets (can also use lagged assets; here use contemporaneous as a simple proxy)
compq['roa'] = (compq['ibq'] / compq['atq']).replace([np.inf, -np.inf], np.nan)

# 8) Gross margin: (revtq - cogsq) / revtq
compq['gross_margin'] = ((compq['revtq'] - compq['cogsq']) / compq['revtq']).replace([np.inf, -np.inf], np.nan)

# 9) Capex-to-assets: capxq / atq
compq['capex_to_assets'] = (compq['capxq'] / compq['atq']).replace([np.inf, -np.inf], np.nan)

# 10) Cash-to-assets: cheq / atq
compq['cash_to_assets'] = (compq['cheq'] / compq['atq']).replace([np.inf, -np.inf], np.nan)

# Keep only what we need going forward
compq_features = compq[[
    'gvkey', 'datadate', 'date',
    'size_log_at', 'bm', 'lev',
    'sales_g_qoq', 'sales_g_yoy', 'asset_g_yoy',
    'roa', 'gross_margin', 'capex_to_assets', 'cash_to_assets'
]].copy()

# (Optional) De-duplicate quarterly rows per gvkey-datadate if needed
compq_features = compq_features.drop_duplicates(subset=['gvkey', 'datadate'])

db.close()

Loading library list...
Done


In [168]:
# ---------------------------
# 4. Merge Data (Permno / GVKEY link needed)
# ---------------------------
# Note: requires CRSP-Compustat linking table (ccmxpf_linktable)
link_table = pd.read_sas(os.path.join('data', 'l4', 'ccmxpf_linktable.sas7bdat'), format='sas7bdat', encoding='utf-8').rename(
    columns={'lpermno': 'permno'})

link_table['linkdt'] = pd.to_datetime(link_table['linkdt'])
link_table['linkenddt'] = pd.to_datetime(link_table['linkenddt'])

# Keep common, high-quality link types and primaries
link_keep_types = {'LC', 'LU'}   # standard/unique CRSP-permno ↔ Compustat-gvkey links
link_keep_prim  = {'P', 'C'}     # prefer primary (P), then consolidated (C)

link_table = link_table.loc[
    link_table['linktype'].isin(link_keep_types) &
    link_table['linkprim'].isin(link_keep_prim),
    ['gvkey', 'permno', 'linkdt', 'linkenddt', 'linktype', 'linkprim']
].copy()

# Treat open-ended linkenddt as far-future to simplify interval filtering
link_table['linkenddt'] = link_table['linkenddt'].fillna(pd.Timestamp('2099-12-31'))

# 4.1 First, join CRSP to link table on permno, then keep only rows where the CRSP month lies within the valid link interval
crsp_lnk = crsp_msf.merge(link_table, on='permno', how='left')
mask_valid_interval = (crsp_lnk['date'] >= crsp_lnk['linkdt']) & (crsp_lnk['date'] <= crsp_lnk['linkenddt'])
crsp_lnk = crsp_lnk.loc[mask_valid_interval].copy()

# If multiple gvkeys match the same (permno, date), keep the "best" one:
#   - Prefer linkprim=P over C; prefer linktype=LC over LU; if tied, keep the most recent linkdt
prim_order = {'P': 0, 'C': 1}
type_order = {'LC': 0, 'LU': 1}
crsp_lnk['prim_rank'] = crsp_lnk['linkprim'].map(prim_order)
crsp_lnk['type_rank'] = crsp_lnk['linktype'].map(type_order)

crsp_lnk = (crsp_lnk
    .sort_values(['permno', 'date', 'prim_rank', 'type_rank', 'linkdt'],
                 ascending=[True, True, True, True, False])
    .drop_duplicates(subset=['permno', 'date'], keep='first')
    .drop(columns=['prim_rank', 'type_rank'])
)



In [169]:
# ==========================================
# Build month and quarter keys; merge CRSP + FF (by month) + CompustatQ (by quarter)
# ==========================================

# --- 0) Normalize dates to pandas Timestamps (if not already) ---
crsp_lnk['date'] = pd.to_datetime(crsp_lnk['date'], errors='coerce')
ff_factors['date'] = pd.to_datetime(ff_factors['date'], errors='coerce')
compq_features['datadate'] = pd.to_datetime(compq_features['datadate'], errors='coerce')
compq_features['date'] = pd.to_datetime(compq_features['date'], errors='coerce')  # quarter end already

# --- 1) Create merge keys ---
# CRSP: month and quarter from the CRSP month-end date
crsp_lnk['datem'] = crsp_lnk['date'].dt.to_period('M')  # YYYY-MM
crsp_lnk['dateq'] = crsp_lnk['date'].dt.to_period('Q')  # YYYYQn

# FF factors: merge on month (datem)
ff_factors['datem'] = ff_factors['date'].dt.to_period('M')  # YYYY-MM

# CompustatQ: create quarter key from the financial report date (prefer datadate’s fiscal quarter)
# Using datadate ensures we do NOT leak future information before the filing quarter.
compq_features['dateq'] = compq_features['datadate'].dt.to_period('Q')  # YYYYQn

# Optional safety: ensure at most one Compustat row per (gvkey, dateq)
compq_q = (compq_features
           .sort_values(['gvkey', 'datadate'])
           .drop_duplicates(subset=['gvkey', 'dateq']))

# --- 2) Merge CRSP with Fama–French by month key (datem) ---
crsp_ff = crsp_lnk.merge(
    ff_factors[['datem', 'mktrf', 'smb', 'hml', 'rf']],
    on='datem',
    how='left',
    validate='many_to_one'  # one factor vector per month
)

# --- 3) Merge CRSP(+FF) with CompustatQ by quarter key (dateq) and gvkey ---
# Requires crsp_lnk to already have gvkey via the CCM link step
final_data = crsp_ff.merge(
    compq_q[['gvkey', 'dateq',
             'size_log_at', 'bm', 'lev',
             'sales_g_qoq', 'sales_g_yoy', 'asset_g_yoy',
             'roa', 'gross_margin', 'capex_to_assets', 'cash_to_assets']],
    on=['gvkey', 'dateq'],
    how='left',
    validate='many_to_one'  # one accounting record per gvkey per quarter
)

# --- 4) Optional: basic cleanup ---
# Drop exact duplicates on (permno, date); keep first occurrence
final_data = final_data.drop_duplicates(subset=['permno', 'date']).copy()

# --- 5) Inspect result ---
print("Final merged shape:", final_data.shape)
print(final_data[['permno','gvkey','date','datem','dateq',
                  'ret','mktrf','smb','hml','rf',
                  'size_log_at','bm','lev']].head())


Final merged shape: (718001, 30)
   permno   gvkey       date    datem   dateq       ret   mktrf     smb  \
0   10001  012994 2010-01-29  2010-01  2010Q1 -0.018932 -0.0335  0.0043   
1   10001  012994 2010-02-26  2010-02  2010Q1 -0.000656  0.0339  0.0118   
2   10001  012994 2010-03-31  2010-03  2010Q1  0.020643   0.063  0.0146   
3   10001  012994 2010-04-30  2010-04  2010Q2  0.124385  0.0199  0.0484   
4   10001  012994 2010-05-28  2010-05  2010Q2  0.004829  -0.079  0.0013   

      hml      rf  size_log_at        bm       lev  
0  0.0033     0.0     4.771168  0.468918  0.350051  
1  0.0318     0.0     4.771168  0.468918  0.350051  
2  0.0219  0.0001     4.771168  0.468918  0.350051  
3  0.0296  0.0001     4.770549  0.467106  0.362329  
4 -0.0248  0.0001     4.770549  0.467106  0.362329  


In [ ]:
# === Preprocessing (monthly, out-of-sample safe) ===
# ---------------------------
# 0) Basic hygiene
# ---------------------------
df = final_data.copy()

# Ensure date sorted
df = df.sort_values(['permno', 'date'])

acct_cols = ['size_log_at','bm','lev','sales_g_qoq','sales_g_yoy','asset_g_yoy',
             'roa','gross_margin','capex_to_assets','cash_to_assets']
for key in ['bm', 'lev', 'sales_g_qoq', 'sales_g_yoy', 'asset_g_yoy',
            'roa', 'gross_margin', 'capex_to_assets', 'cash_to_assets']:
    df[key] = df[key].astype(np.float64)

# Keep core columns presence
needed_cols = ['permno','date','ret','rf','me','mktrf','smb','hml'] + acct_cols

for c in needed_cols:
    if c not in df.columns:
        raise ValueError(f"Missing required column: {c}. Please check data loading step.")

# ---------------------------
# 1) Target: next-month excess return
# ---------------------------
df['excess_ret'] = df['ret'] - df['rf']  # current month excess return
df['excess_ret_fwd1'] = df.groupby('permno')['excess_ret'].shift(-1)  # predict next month

# ---------------------------
# 2) Feature engineering (all lagged to avoid look-ahead)
#    - Size (log ME) already as 'size' from Compustat; keep a CRSP-based log(ME) too for robustness
#    - Book-to-Market 'bm' from Compustat
#    - Momentum 12-2 (skip most recent month): sum of returns t-12..t-2
#    - Volatility 6m: rolling std of monthly returns
#    - Past market factor (lagged): mktrf_{t-1}
# ---------------------------

# CRSP-based log(ME)
df['log_me'] = np.log(df['me'].replace({0: np.nan}))

# Momentum 12-2:
# comp_12 = (1+ret).rolling(12).prod() - 1
# then subtract last month's return (t-1) to "skip" the most recent month
comp_12 = df.groupby('permno')['ret'].transform(
    lambda x: (1 + x).rolling(12).apply(np.prod, raw=True) - 1
)
last_m = df.groupby('permno', sort=False)['ret'].shift(1)
df['mom_12_2'] = comp_12 - last_m

# Volatility 6m (rolling std over last 6 months)
df['vol_6m'] = df.groupby('permno', sort=False)['ret'] \
                 .transform(lambda x: x.rolling(6).std())

# Lagged market factor (last month, global series)
df = df.sort_values(['date'])  # ensure chronological order for global lag
df['mktrf_lag1'] = df['mktrf'].shift(1)
df['smb_lag1']   = df['smb'].shift(1)
df['hml_lag1']   = df['hml'].shift(1)
df['rf_lag1']    = df['rf'].shift(1)

# Make sure fundamentals are carried forward within each stock (or use gvkey if preferred)
df = df.sort_values(['permno', 'date'])
df[['size_log_at', 'bm']] = df.groupby('permno')[['size_log_at', 'bm']].ffill()


# ---------------------------
# 3) Assemble feature set
# ---------------------------
feature_cols = [
    # CRSP-derived
    'log_me', 'mom_12_2', 'vol_6m',
    # FF factors
    'mktrf', 'smb', 'hml', 'rf',
    # Compustat quarterly fundamentals (carried forward)
    'size_log_at', 'bm', 'lev', 'sales_g_qoq', 'sales_g_yoy',
    'asset_g_yoy', 'roa', 'gross_margin', 'capex_to_assets', 'cash_to_assets'
]

# Drop rows where core target or features missing
cols_needed_for_model = ['permno','date','excess_ret_fwd1'] + feature_cols
df_model = df[cols_needed_for_model].replace([np.inf, -np.inf], np.nan).dropna(how='any')

# ---------------------------
# 4) Winsorization (1% / 99%) on features to reduce tail sensitivity
# ---------------------------
def winsorize_df(dfin, cols, p=0.01):
    d = dfin.copy()
    for c in cols:
        q_low, q_high = d[c].quantile([p, 1-p])
        d[c] = d[c].clip(lower=q_low, upper=q_high)
    return d

df_model = winsorize_df(df_model, feature_cols, p=0.01)
df_model['target'] = df_model['excess_ret_fwd1']

# ---------------------------
# 5) Temporal train/test split (expandable; here: fixed split)
#    Train: 2005-01-01 ~ 2017-12-31
#    Test : 2018-01-01 ~ 2024-12-31
# ---------------------------
train_end = pd.Timestamp('2017-12-31')
test_start = pd.Timestamp('2018-01-01')

train_mask = df_model['date'] <= train_end
test_mask  = df_model['date'] >= test_start

train_df = df_model.loc[train_mask].dropna(how='any')
test_df  = df_model.loc[test_mask].dropna(how='any')

# # Shuffle is NOT used (time-series); keep order if needed for rolling evaluation.
# # Basic checks
# print(f"Train rows: {len(train_df):,} | Test rows: {len(test_df):,}")
# print(f"Train period: {train_df['date'].min().date()} ~ {train_df['date'].max().date()}")
# print(f"Test  period: {test_df['date'].min().date()} ~ {test_df['date'].max().date()}")

# X_train = train_df[feature_cols].to_numpy(dtype='float64')
# y_train = train_df['target'].to_numpy(dtype='float64')
# X_test  = test_df[feature_cols].to_numpy(dtype='float64')
# y_test  = test_df['target'].to_numpy(dtype='float64')

# print("Train:", train_df['date'].min(), "~", train_df['date'].max(), len(train_df))
# print("Test :", test_df['date'].min(), "~", test_df['date'].max(), len(test_df))


Train rows: 313,729 | Test rows: 123,655
Train period: 2010-12-31 ~ 2017-12-29
Test  period: 2018-01-31 ~ 2020-11-30
Train: 2010-12-31 00:00:00 ~ 2017-12-29 00:00:00 313729
Test : 2018-01-31 00:00:00 ~ 2020-11-30 00:00:00 123655


In [171]:
X = df_model[feature_cols].to_numpy(dtype='float64')
y = df_model['excess_ret_fwd1'].to_numpy(dtype='float64')

# 2) Standardize features (recommended for linear models)
scaler = StandardScaler()
X_std = scaler.fit_transform(X)


In [156]:
from math import sqrt
try:
    from scipy.stats import norm
    _HAS_SCIPY = True
except Exception:
    _HAS_SCIPY = False
    
def _nw_var(d, lag=0):
    """
    Newey–West variance of the mean of d (loss differential), lag = q.
    Returns var(mean(d)) = (gamma0 + 2*sum w_k*gamma_k)/T
    """
    d = np.asarray(d, float)
    T = len(d)
    d = d - d.mean()
    gamma0 = np.dot(d, d) / T
    var = gamma0
    for k in range(1, lag + 1):
        w = 1.0 - k / (lag + 1.0)  # Bartlett weights
        cov = np.dot(d[k:], d[:-k]) / T
        var += 2.0 * w * cov
    return var / T  # variance of the mean

def dm_test(y_true, y_model, y_bench, lag=0):
    """
    Diebold–Mariano test (squared-error loss) comparing model vs benchmark.
    H0: E(loss_model - loss_bench) = 0  (no improvement)
    Returns: (stat, p_two_sided, p_one_sided) where one-sided favors the model.
    """
    e_m = y_true - y_model
    e_b = y_true - y_bench
    d = (e_b**2) - (e_m**2)  # positive => model improves over benchmark
    m = np.mean(d)
    v = _nw_var(d, lag=lag)
    stat = m / sqrt(v) if v > 0 else np.nan
    if _HAS_SCIPY:
        p_two = 2.0 * (1.0 - norm.cdf(abs(stat)))
        p_one = 1.0 - norm.cdf(stat)  # H1: model better (stat > 0)
    else:
        # simple normal approx via error function
        from math import erf
        def _phi(z): return 0.5 * (1.0 + erf(z / np.sqrt(2.0)))
        p_two = 2.0 * (1.0 - _phi(abs(stat)))
        p_one = 1.0 - _phi(stat)
    return stat, p_two, p_one

def clark_west_test(y_true, f_model, f_bench, lag=0):
    """
    Clark–West (2007) test for nested models vs benchmark (squared-error loss).
    Let e_m = y - f_model, e_b = y - f_bench.
    Adjusted differential: d_t = e_b^2 - ( e_m^2 - (f_b - f_m)^2 )
    H0: E(d_t) = 0 (no improvement); H1: E(d_t) > 0 (model improves).
    Returns: (stat, p_one_sided)
    """
    e_m = y_true - f_model
    e_b = y_true - f_bench
    d = (e_b**2) - (e_m**2 - (f_bench - f_model)**2)
    m = np.mean(d)
    v = _nw_var(d, lag=lag)
    stat = m / sqrt(v) if v > 0 else np.nan
    if _HAS_SCIPY:
        p_one = 1.0 - norm.cdf(stat)
    else:
        from math import erf
        def _phi(z): return 0.5 * (1.0 + erf(z / np.sqrt(2.0)))
        p_one = 1.0 - _phi(stat)
    return stat, p_one

In [157]:
from sklearn.base import clone

def evaluate_model_rolling(
    model,
    X,
    y,
    dates=None,
    train_window=120,     # number of observations in the training window (for rolling)
    test_horizon=1,       # predict the next H observations each step (H=1 is most common)
    step=1,               # how many observations to move the window forward each iteration
    scaler=None,          # optional sklearn transformer, e.g., StandardScaler()
    model_name=None,
    hac_lag=0,            # Newey–West lag for DM/CW tests (0 for non-overlapping horizons)
    expanding=False,      # True = expanding window; False = fixed-length rolling window
    dropna=True           # drop windows with NaNs in train/test
):
    """
    Rolling / Expanding out-of-sample evaluation for a predictive model.

    Workflow per step:
      1) Fit the model on the training window (rolling or expanding).
      2) Predict the next `test_horizon` observations.
      3) Build a benchmark using the training-window historical mean.
      4) Accumulate predictions/targets across all steps.
      5) Compute performance metrics and tests on the concatenated OOS sample.

    Parameters
    ----------
    model : sklearn-compatible estimator
        Must implement fit(X, y) and predict(X).
    X : array-like or DataFrame, shape (T, K)
        Features sorted in chronological order by rows (oldest to newest).
    y : array-like, shape (T,)
        Target aligned to X (y[t] corresponds to X[t, :]).
    dates : array-like or DatetimeIndex, optional
        Timestamps for rows of X/y (used in the detailed return).
        If None and X is a DataFrame with a DatetimeIndex, that index will be used.
    train_window : int
        Size of training window for rolling (ignored if expanding=True).
    test_horizon : int
        Number of future observations to predict per step (typical choice is 1).
    step : int
        Window shift size between iterations.
    scaler : sklearn transformer, optional
        If provided, it will be fit on each training window and applied to both train/test.
    model_name : str, optional
        Label for the output table. Defaults to estimator class name.
    hac_lag : int
        Newey–West lag for DM and Clark–West tests (set >0 if horizons overlap).
    expanding : bool
        If True, uses expanding window; otherwise uses fixed-size rolling window.
    dropna : bool
        If True, drops NaNs inside each train/test window before fitting/evaluating.

    Returns
    -------
    results : pd.DataFrame
        Single-row summary with RMSE, MAE, HitRatio, R2_OOS, DM_stat, CW_stat,
        number of fits, total/mean fit time.
    detailed : pd.DataFrame
        Row-by-row OOS records with columns: ['date', 'y', 'y_hat', 'y_bench'].
    """

    # ---- Convert inputs to numpy arrays; capture dates ----
    if isinstance(X, (pd.DataFrame, pd.Series)):
        X_arr = X.values
    else:
        X_arr = np.asarray(X)

    y_arr = np.asarray(y).reshape(-1,)
    T = len(y_arr)

    if dates is not None:
        dates_arr = np.asarray(dates)
    elif isinstance(X, pd.DataFrame) and isinstance(X.index, pd.DatetimeIndex):
        dates_arr = X.index.to_numpy()
    else:
        dates_arr = np.arange(T)  # fallback index

    # ---- Containers for OOS predictions and bookkeeping ----
    preds_all, y_all, bench_all, ts_all = [], [], [], []
    fit_times = []
    n_fits = 0

    # Starting index for the first evaluation step
    # For expanding: start with at least 1 obs; for rolling: need `train_window` obs
    start_idx = 1 if expanding else train_window

    # Loop over windows
    for i in range(start_idx, T - test_horizon + 1, step):
        # Training slice
        tr_start = 0 if expanding else i - train_window
        tr_end   = i                  # train on [tr_start, tr_end)
        te_start = i
        te_end   = i + test_horizon   # test on [te_start, te_end)

        X_tr, y_tr = X_arr[tr_start:tr_end, :], y_arr[tr_start:tr_end]
        X_te, y_te = X_arr[te_start:te_end,   :], y_arr[te_start:te_end]
        te_dates   = dates_arr[te_start:te_end]

        # Optional NaN handling within each window
        if dropna:
            tr_mask = np.isfinite(y_tr)
            if tr_mask.sum() < 3:
                continue
            X_tr, y_tr = X_tr[tr_mask, :], y_tr[tr_mask]

            te_mask = np.isfinite(y_te)
            if te_mask.sum() == 0:
                continue
            X_te, y_te = X_te[te_mask, :], y_te[te_mask]
            te_dates   = te_dates[te_mask]

        # Optional scaling: fit scaler on the training window only
        if scaler is not None:
            sc = clone(scaler)
            X_tr_s = sc.fit_transform(X_tr)
            X_te_s = sc.transform(X_te)
        else:
            X_tr_s, X_te_s = X_tr, X_te

        # Clone model to ensure independent refit each step
        mdl = clone(model)

        # Fit and time it
        t0 = time.time()
        mdl.fit(X_tr_s, y_tr)
        fit_times.append(time.time() - t0)
        n_fits += 1

        # Predict on the test slice
        y_hat = np.asarray(mdl.predict(X_te_s)).reshape(-1,)

        # Benchmark: historical mean from the *training* window
        mu_hat = float(np.nanmean(y_tr))
        y_bmk  = np.full_like(y_te, mu_hat, dtype=float)

        # Accumulate
        preds_all.append(y_hat)
        y_all.append(y_te)
        bench_all.append(y_bmk)
        ts_all.append(te_dates)

    # Concatenate across all steps
    if len(y_all) == 0:
        raise ValueError("No test observations produced. Check window sizes or data length.")

    y_pred  = np.concatenate(preds_all)
    y_true  = np.concatenate(y_all)
    y_bench = np.concatenate(bench_all)
    t_out   = np.concatenate(ts_all)

    # ---- Metrics on the concatenated OOS sample ----
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    mae  = float(mean_absolute_error(y_true, y_pred))
    hit  = float((np.sign(y_pred) == np.sign(y_true)).mean())

    sse_model = float(np.sum((y_true  - y_pred)  ** 2))
    sse_bench = float(np.sum((y_true  - y_bench) ** 2))
    r2_oos = 1.0 - sse_model / sse_bench if sse_bench > 0 else np.nan

    # DM (squared-error loss) and Clark–West (nested models) vs benchmark
    dm_stat, dm_p_two, dm_p_one = dm_test(y_true, y_pred, y_bench, lag=hac_lag)
    cw_stat, cw_p_one = clark_west_test(y_true, y_pred, y_bench, lag=hac_lag)

    # Fit-time accounting
    fit_time_total = float(np.sum(fit_times)) if fit_times else 0.0
    fit_time_mean  = float(np.mean(fit_times)) if fit_times else 0.0

    # One-row summary for pretty printing
    results = pd.DataFrame([{
        "Model": model_name or model.__class__.__name__,
        "RMSE": round(rmse, 6),
        "MAE": round(mae, 6),
        "HitRatio": round(hit, 4),
        "R2_OOS": round(r2_oos, 6),
        "DM_stat": round(dm_stat, 3),
        "CW_stat": round(cw_stat, 3),
        "Fits": int(n_fits),
        "FitTime_Total (s)": round(fit_time_total, 4),
        "FitTime_Mean (s)": round(fit_time_mean, 4),
    }])

    # Row-level OOS records for plotting and segment analysis
    detailed = pd.DataFrame({
        "date": t_out,
        "y": y_true,
        "y_hat": y_pred,
        "y_bench": y_bench
    }).sort_values("date").reset_index(drop=True)

    return results, detailed


## Ordinary Least Squares (OLS)

**Idea:**  
Ordinary Least Squares (OLS) is the baseline linear regression method. It estimates the relationship between a dependent variable $y$ and explanatory variables $X$ by fitting a straight line (or hyperplane) that minimizes the **sum of squared errors**.

**Model:**
$$
y_i = \alpha + X_i^\top \beta + \varepsilon_i
$$

- $y_i$: outcome (e.g., next-month excess return)  
- $X_i$: vector of predictors (firm characteristics, factors)  
- $\beta$: coefficients to be estimated  
- $\varepsilon_i$: error term (unexplained part)

**Estimation Principle:**  
OLS chooses $\hat{\beta}$ to minimize
$$
\min_{\beta} \sum_{i=1}^n \left(y_i - X_i^\top \beta \right)^2
$$

**Interpretation:**  
- Each $\beta_j$ shows how a one-unit change in predictor $X_j$ affects the expected outcome, holding others constant.  
- Provides a simple and interpretable baseline.  

**Limitations:**  
- Assumes linear relationships.  
- Sensitive to multicollinearity and outliers.  
- May overfit when predictors are high-dimensional and noisy.

In [139]:
# 1) Build matrices
# X_train = train_df[feature_cols].to_numpy(dtype='float64')
# y_train = train_df['excess_ret_fwd1'].to_numpy(dtype='float64')
# X_test  = test_df[feature_cols].to_numpy(dtype='float64')
# y_test  = test_df['excess_ret_fwd1'].to_numpy(dtype='float64')

# 2) Standardize features (recommended for linear models)
scaler = StandardScaler()
X_train_std = scaler.fit_transform(X_train)
X_test_std  = scaler.transform(X_test)

# 3) Fit and evaluate OLS
ols = LinearRegression()
ols_results, ols_model = evaluate_model(ols, X_train_std, y_train, X_test_std, y_test,
                            model_name="OLS", hac_lag=0)

# 4) Print evaluation summary
print("=== OLS Test Results ===")
display(ols_results)

all_results = []
all_results.append(ols_results)

=== OLS Test Results ===


,Model,RMSE,MAE,HitRatio,FitTime (s),R2_OOS,DM_stat,DM_p(two),DM_p(one),CW_stat,CW_p(one)
0,OLS,0.047272,0.03595,0.7021,0.002,-0.020472,-0.758,0.4486,0.7757,0.141,0.4441


## Regularized Regression: Ridge, Lasso, and Elastic Net

**Motivation:**  
Ordinary Least Squares (OLS) can overfit when predictors are **high-dimensional**, **noisy**, or **highly correlated**.  
Regularization methods add a penalty to the loss function, shrinking coefficients to improve generalization.

---

### Lasso Regression (L1 penalty)
Lasso uses the **sum of absolute coefficients** as a penalty:
$$
\min_{\beta} \sum_{i=1}^n (y_i - X_i^\top \beta)^2 + \lambda \sum_{j=1}^p |\beta_j|
$$

- Performs **variable selection**: some coefficients shrink exactly to zero.  
- Useful when we believe only a subset of predictors are truly important.  
- Tends to select one variable from a group of correlated variables.


---

### Ridge Regression (L2 penalty)
Ridge shrinks coefficients by adding the **sum of squared coefficients** as a penalty:
$$
\min_{\beta} \sum_{i=1}^n (y_i - X_i^\top \beta)^2 + \lambda \sum_{j=1}^p \beta_j^2
$$

- Keeps all predictors, but shrinks them toward zero.  
- Works well when predictors are correlated.  
- Tuning parameter $\lambda$ controls the amount of shrinkage.
---

### Elastic Net (L1 + L2 penalty)
Elastic Net combines both L1 and L2 penalties:
$$
\min_{\beta} \sum_{i=1}^n (y_i - X_i^\top \beta)^2 + \lambda_1 \sum_{j=1}^p |\beta_j| + \lambda_2 \sum_{j=1}^p \beta_j^2
$$

- Balances **shrinkage** (Ridge) and **variable selection** (Lasso).  
- Works better than Lasso when predictors are highly correlated.  
- Controlled by two hyperparameters: $\lambda_1$ (L1 strength) and $\lambda_2$ (L2 strength).

---

**Takeaway:**  
- **Lasso** → performs automatic feature selection.  
- **Ridge** → keeps all predictors, reduces variance.  
- **Elastic Net** → compromise, often best in practice.


In [140]:
# === Lasso Regression ===
# 1) Choose alpha via CV (no shuffle to respect time ordering)
alphas = np.logspace(-4, 1, 40)  # 1e-4 … 10
lasso_cv = LassoCV(alphas=alphas, cv=5, fit_intercept=True, max_iter=5000, n_jobs=-1, random_state=42)
lasso_cv.fit(X_train_std, y_train)
best_alpha = float(lasso_cv.alpha_)
print(f"[LassoCV] best alpha = {best_alpha:.6g}")

# 2) Refit Lasso with best alpha (so fit time is comparable to OLS)
lasso = Lasso(alpha=best_alpha, fit_intercept=True, max_iter=5000, random_state=42)
lasso_results, lasso_model = evaluate_model(
    lasso, X_train_std, y_train, X_test_std, y_test, model_name=f"Lasso(alpha={best_alpha:.3g})", hac_lag=0
)

print("=== Lasso Test Results ===")
display(lasso_results)


[LassoCV] best alpha = 10
=== Lasso Test Results ===


,Model,RMSE,MAE,HitRatio,FitTime (s),R2_OOS,DM_stat,DM_p(two),DM_p(one),CW_stat,CW_p(one)
0,Lasso(alpha=10),0.046795,0.036413,0.7128,0.001,0.0,NaN,NaN,NaN,NaN,NaN


In [172]:
# === Lasso Regression with Rolling Evaluation ===

# 1) Choose alpha via CV (no shuffle to respect time ordering)
alphas = np.logspace(-4, 1, 40)  # 1e-4 … 10
lasso_cv = LassoCV(
    alphas=alphas, 
    cv=5, 
    fit_intercept=True, 
    max_iter=5000, 
    n_jobs=-1, 
    random_state=42
)
lasso_cv.fit(X_std, y)
best_alpha = float(lasso_cv.alpha_)
print(f"[LassoCV] best alpha = {best_alpha:.6g}")

# 2) Define Lasso with best alpha
lasso = Lasso(alpha=best_alpha, fit_intercept=True, max_iter=5000, random_state=42)

# 3) Rolling evaluation (example: 60-month rolling window, 12-month test window)
lasso_results, lasso_details = evaluate_model_rolling(
    model=lasso,
    X=X_std,              # full standardized features
    y=y,                  # full target returns
    train_window=60,       # 5 years for training
    test_horizon=12,       # 1 year for testing
    model_name=f"Lasso(alpha={best_alpha:.3g})",
    hac_lag=0
)

print("=== Lasso Rolling Window Results ===")
display(lasso_results)


[LassoCV] best alpha = 0.0001


KeyboardInterrupt: 

In [141]:

# 3) Inspect sparsity and coefficients (standardized features)
coef = lasso_model.coef_
nz_mask = coef != 0
n_nz = int(nz_mask.sum())
print(f"Nonzero coefficients: {n_nz} / {len(coef)}")

coef_df = pd.DataFrame({
    "feature": feature_cols,
    "coef_std": coef
}).sort_values("coef_std", key=lambda s: s.abs(), ascending=False)

print("\n=== Lasso Coefficients (standardized) ===")
print(coef_df.to_string(index=False))

# 4) Store results
all_results.append(lasso_results)

Nonzero coefficients: 0 / 7

=== Lasso Coefficients (standardized) ===
    feature  coef_std
      mktrf       0.0
        smb       0.0
        hml      -0.0
 gs10_yield      -0.0
term_spread      -0.0
  div_yield       0.0
 def_spread      -0.0


In [142]:
# === Ridge Regression ===

# 1) Choose alpha via CV
alphas = np.logspace(-4, 4, 50)  # from 1e-4 to 1e4
ridge_cv = RidgeCV(alphas=alphas, store_cv_values=False)
ridge_cv.fit(X_train_std, y_train)
best_alpha = float(ridge_cv.alpha_)
print(f"[RidgeCV] best alpha = {best_alpha:.6g}")

# 4) Refit Ridge with best alpha
ridge = Ridge(alpha=best_alpha, fit_intercept=True, random_state=42)
ridge_results, ridge_model = evaluate_model(
    ridge, X_train_std, y_train, X_test_std, y_test, model_name=f"Ridge(alpha={best_alpha:.3g})", hac_lag=0
)

print("=== Ridge Test Results ===")
display(ridge_results)


[RidgeCV] best alpha = 10000
=== Ridge Test Results ===


d:\anaconda3\envs\finnlp\Lib\site-packages\sklearn\linear_model\_ridge.py:2385: FutureWarning: 'store_cv_values' is deprecated in version 1.5 and will be removed in 1.7. Use 'store_cv_results' instead.
  warnings.warn(


,Model,RMSE,MAE,HitRatio,FitTime (s),R2_OOS,DM_stat,DM_p(two),DM_p(one),CW_stat,CW_p(one)
0,Ridge(alpha=1e+04),0.046803,0.036397,0.7128,0.005,-0.000325,-0.586,0.5578,0.7211,-0.562,0.7129


In [143]:

# 2) Print coefficients (standardized)
coef_df = pd.DataFrame({
    "feature": feature_cols,
    "coef_std": ridge_model.coef_
}).sort_values("coef_std", key=lambda s: s.abs(), ascending=False)

print("\n=== Ridge Coefficients (standardized) ===")
print(coef_df.to_string(index=False))

# 3) Store results
all_results.append(ridge_results)


=== Ridge Coefficients (standardized) ===
    feature  coef_std
  div_yield  0.000095
      mktrf  0.000091
 def_spread -0.000075
        hml -0.000050
 gs10_yield -0.000045
        smb  0.000037
term_spread -0.000002


In [144]:
# === Elastic Net Regression ===
# 1) Hyperparameter search (CV) for alpha and l1_ratio
alphas = np.logspace(-4, 1, 40)                 # 1e-4 … 10
l1_grid = [0.1, 0.3, 0.5, 0.7, 0.9]             # blend between Ridge(0) and Lasso(1)
enet_cv = ElasticNetCV(
    alphas=alphas,
    l1_ratio=l1_grid,
    cv=5,
    max_iter=10000,
    n_jobs=-1,
    random_state=42,
    fit_intercept=True
)
enet_cv.fit(X_train_std, y_train)

best_alpha = float(enet_cv.alpha_)
best_l1    = float(enet_cv.l1_ratio_)
print(f"[ElasticNetCV] best alpha = {best_alpha:.6g}, best l1_ratio = {best_l1:.3g}")

# 2) Refit Elastic Net using the best params (for fair FitTime measurement)
enet = ElasticNet(
    alpha=best_alpha,
    l1_ratio=best_l1,
    max_iter=10000,
    fit_intercept=True,
    random_state=42
)

enet_results, enet_model = evaluate_model(
    enet, X_train_std, y_train, X_test_std, y_test,
    model_name=f"ElasticNet(alpha={best_alpha:.3g}, l1={best_l1:.2g})"
)

print("=== Elastic Net Test Results ===")
display(enet_results)


[ElasticNetCV] best alpha = 10, best l1_ratio = 0.1
=== Elastic Net Test Results ===


,Model,RMSE,MAE,HitRatio,FitTime (s),R2_OOS,DM_stat,DM_p(two),DM_p(one),CW_stat,CW_p(one)
0,"ElasticNet(alpha=10, l1=0.1)",0.046795,0.036413,0.7128,0.001,0.0,NaN,NaN,NaN,NaN,NaN


In [145]:

# 3) Coefficient inspection (standardized features)
coef = enet_model.coef_
nz_mask = coef != 0
print(f"Nonzero coefficients: {int(nz_mask.sum())} / {len(coef)}")

coef_df = pd.DataFrame({
    "feature": feature_cols,
    "coef_std": coef
}).sort_values("coef_std", key=lambda s: s.abs(), ascending=False)

print("\n=== Elastic Net Coefficients (standardized) ===")
print(coef_df.to_string(index=False))

# 4) Store results for later comparison
all_results.append(enet_results)

Nonzero coefficients: 0 / 7

=== Elastic Net Coefficients (standardized) ===
    feature  coef_std
      mktrf       0.0
        smb       0.0
        hml      -0.0
 gs10_yield      -0.0
term_spread      -0.0
  div_yield       0.0
 def_spread      -0.0


## Random Forest: Intuition and Application

**Random Forest (RF)** is an ensemble learning method introduced by Breiman (2001).  
It builds on decision trees but improves stability and accuracy through **bagging** and **random feature selection**.

- **Decision Tree Basics**:  
  Splits data recursively into regions based on predictor values.  
  Simple and interpretable, but prone to **overfitting** and high variance.

- **Random Forest Mechanics**:  
  - **Bagging**: Each tree is trained on a bootstrap sample of the data.  
  - **Feature Subsampling**: At each split, only a random subset of features is considered.  
  - **Ensemble Prediction**:  
    - Regression: average of predictions across trees.  
    - Classification: majority vote across trees.  

- **Strengths**:  
  - Captures **nonlinearities** and **interactions** automatically.  
  - Robust to noise and overfitting by averaging many decorrelated trees.  
  - Scales well to high-dimensional datasets; training is parallelizable.  
  - Provides **feature importance** measures useful in finance.

- **Weaknesses**:  
  - Acts like a **black box**; less interpretable than OLS or Lasso.  
  - May be biased toward strong predictors if not tuned carefully.  
  - Less efficient for sparse, very high-dimensional data.

**Finance Takeaway**:  
Random Forest is a strong off-the-shelf model for stock return prediction, especially when relationships are **nonlinear** and **noisy**.

In [148]:
# 1) Define RF model (sensible defaults for classroom demo)
n_estimaters = 300
rf = RandomForestRegressor(
    n_estimators=n_estimaters,
    max_depth=None,          # let trees grow; adjust if overfitting
    min_samples_leaf=1,
    max_features='sqrt',     # typical for tabular data
    bootstrap=True,
    n_jobs=-1,
    random_state=42
)

# 2) Fit & evaluate
rf_results, rf_model = evaluate_model(
    rf, X_train, y_train, X_test, y_test, model_name=f"RandomForest(n={n_estimaters})"
)

print("=== Random Forest Test Results ===")
display(rf_results)

=== Random Forest Test Results ===


,Model,RMSE,MAE,HitRatio,FitTime (s),R2_OOS,DM_stat,DM_p(two),DM_p(one),CW_stat,CW_p(one)
0,RandomForest(n=300),0.049196,0.037725,0.6064,0.2782,-0.105252,-2.232,0.0256,0.9872,-1.474,0.9298


In [149]:
# 3) Feature importances
fi = pd.DataFrame({
    "feature": feature_cols,
    "importance": rf_model.feature_importances_
}).sort_values("importance", ascending=False)

print("\n=== Random Forest Feature Importances ===")
print(fi.to_string(index=False))

# 4) Store results
all_results.append(rf_results)


=== Random Forest Feature Importances ===
    feature  importance
      mktrf    0.161123
        hml    0.157534
 def_spread    0.152966
  div_yield    0.145641
        smb    0.141933
 gs10_yield    0.125534
term_spread    0.115268


## Gradient Boosting Variants: XGBoost and LightGBM

**Gradient Boosting** builds trees sequentially, where each new tree corrects the errors of the previous ones.  
Among its modern implementations, **XGBoost** and **LightGBM** are industry standards for structured/tabular data.

---

### 🔹 XGBoost (Chen & Guestrin, 2016)
- **Growth Strategy**: Level-wise (splits nodes layer by layer).
- **Strengths**:
  - Strong regularization (L1/L2 penalties) → helps prevent overfitting.
  - Stable and widely used in Kaggle competitions and finance.
  - Flexible with custom loss functions and objectives.
- **Limitations**:
  - Training can be slower on very large datasets compared to LightGBM.

---

### 🔹 LightGBM (Ke et al., 2017)
- **Growth Strategy**: Leaf-wise with depth constraints.
- **Strengths**:
  - Very **fast and memory-efficient** (histogram-based splitting).
  - Handles large datasets and high-dimensional features effectively.
  - Supports categorical variables natively.
- **Limitations**:
  - Leaf-wise growth may **overfit** small datasets if depth is not controlled.
  - Requires careful tuning of learning rate, max depth, and regularization.

---

### Comparison
| Feature               | XGBoost                     | LightGBM                       |
|-----------------------|-----------------------------|--------------------------------|
| Tree Growth           | Level-wise                  | Leaf-wise                      |
| Speed & Memory        | Moderate                    | Very fast, memory-efficient    |
| Regularization        | Strong (L1/L2)              | Relies more on depth control   |
| Robustness            | Stable, widely adopted      | Faster but risk of overfitting |
| Use Case in Finance   | Robust baseline             | Large-scale, high-frequency data|

---

**Finance Takeaway**:  
Both are strong choices for **stock return prediction**.  
- Use **XGBoost** for stability and interpretability.  
- Use **LightGBM** when dataset is **large and high-dimensional** with speed requirements.


In [150]:
# === XGBoost Regression ===
# 1) Define model (robust classroom defaults)
xgb = XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,           # L2
    reg_alpha=0.0,            # L1 (set >0 to encourage sparsity)
    objective="reg:squarederror",
    n_jobs=-1,
    random_state=42,
    tree_method="hist"        # fast & memory-efficient
)

# 2) Fit & evaluate
xgb_results, xgb_model = evaluate_model(
    xgb, X_train, y_train, X_test, y_test, model_name="XGBoost(500, lr=0.05)"
)

print("=== XGBoost Test Results ===")
display(xgb_results)

=== XGBoost Test Results ===


,Model,RMSE,MAE,HitRatio,FitTime (s),R2_OOS,DM_stat,DM_p(two),DM_p(one),CW_stat,CW_p(one)
0,"XGBoost(500, lr=0.05)",0.050096,0.037929,0.5745,0.3712,-0.146057,-1.611,0.1071,0.9464,-0.17,0.5674


In [151]:
# 3) Feature importances
xgb_fi = pd.DataFrame({
    "feature": feature_cols,
    "importance": xgb_model.feature_importances_
}).sort_values("importance", ascending=False)

print("\n=== XGBoost Feature Importances ===")
print(xgb_fi.to_string(index=False))

# 4) Store results
all_results.append(xgb_results)


=== XGBoost Feature Importances ===
    feature  importance
 gs10_yield    0.176990
 def_spread    0.173125
  div_yield    0.144114
        hml    0.136003
      mktrf    0.132355
        smb    0.121767
term_spread    0.115646


In [152]:
# === LightGBM Regression ===
# 1) Define model (fast classroom defaults)
lgbm = lgb.LGBMRegressor(
    n_estimators=600,
    learning_rate=0.05,
    num_leaves=31,
    max_depth=-1,            # no explicit depth limit; controlled by num_leaves
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    reg_alpha=0.0,
    random_state=42,
    n_jobs=-1
)

# 2) Fit & evaluate
lgbm_results, lgbm_model = evaluate_model(
    lgbm, X_train, y_train, X_test, y_test, model_name="LightGBM(600, lr=0.05)"
)

print("=== LightGBM Test Results ===")
display(lgbm_results)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000205 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 618
[LightGBM] [Info] Number of data points in the train set: 287, number of used features: 7
[LightGBM] [Info] Start training from score 0.006022
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

d:\anaconda3\envs\finnlp\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


,Model,RMSE,MAE,HitRatio,FitTime (s),R2_OOS,DM_stat,DM_p(two),DM_p(one),CW_stat,CW_p(one)
0,"LightGBM(600, lr=0.05)",0.051817,0.040891,0.5532,0.3247,-0.226146,-1.818,0.069,0.9655,0.2,0.4206


In [153]:
# 3) Feature importances (gain-based is often more informative than split-count)
lgbm_fi = pd.DataFrame({
    "feature": feature_cols,
    "importance": lgbm_model.booster_.feature_importance(importance_type='gain')
}).sort_values("importance", ascending=False)

print("\n=== LightGBM Feature Importances (gain) ===")
print(lgbm_fi.to_string(index=False))

# 4) Store results
all_results.append(lgbm_results)


=== LightGBM Feature Importances (gain) ===
    feature  importance
      mktrf    0.909059
  div_yield    0.772846
 gs10_yield    0.743791
        hml    0.654102
 def_spread    0.641197
        smb    0.619286
term_spread    0.470007


In [154]:
print("=== Summary of All Model Results ===")
final_summary = pd.concat(all_results, ignore_index=True)
display(final_summary)

=== Summary of All Model Results ===


,Model,RMSE,MAE,HitRatio,FitTime (s),R2_OOS,DM_stat,DM_p(two),DM_p(one),CW_stat,CW_p(one)
0,OLS,0.047272,0.035950,0.7021,0.0020,-0.020472,-0.758,0.4486,0.7757,0.141,0.4441
1,Lasso(alpha=10),0.046795,0.036413,0.7128,0.0010,0.000000,NaN,NaN,NaN,NaN,NaN
2,Ridge(alpha=1e+04),0.046803,0.036397,0.7128,0.0050,-0.000325,-0.586,0.5578,0.7211,-0.562,0.7129
3,"ElasticNet(alpha=10, l1=0.1)",0.046795,0.036413,0.7128,0.0010,0.000000,NaN,NaN,NaN,NaN,NaN
4,RandomForest(n=100),0.049449,0.038128,0.5851,0.0980,-0.116634,-2.429,0.0152,0.9924,-1.530,0.9370
5,RandomForest(n=300),0.049196,0.037725,0.6064,0.2782,-0.105252,-2.232,0.0256,0.9872,-1.474,0.9298
6,"XGBoost(500, lr=0.05)",0.050096,0.037929,0.5745,0.3712,-0.146057,-1.611,0.1071,0.9464,-0.170,0.5674
7,"LightGBM(600, lr=0.05)",0.051817,0.040891,0.5532,0.3247,-0.226146,-1.818,0.0690,0.9655,0.200,0.4206
